In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [5]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

In [6]:
data_fake.head(6)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"


In [7]:
data_true.head(6)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"


In [8]:
data_fake["class"] = 0
data_true["class"] = 1

In [9]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [10]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470,-1):
    data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406,-1):
    data_true.drop([i], axis=0,inplace=True)

In [11]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\surya\AppData\Local\Temp\ipykernel_7264\277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\surya\AppData\Local\Temp\ipykernel_7264\277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [12]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [13]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [14]:
data = data_merge.drop(['title','subject', 'date'], axis = 1)

In [15]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [16]:
data = data.sample(frac=1)

In [17]:
data

,text,class
10570,"WASHINGTON (Reuters) - One evening last June, ...",1
20414,MOSCOW (Reuters) - Russian President Vladimir ...,1
4137,Just when you thought Republicans couldn t get...,0
10174,Wow! Open threats that government will kill ...,0
2913,"WASHINGTON (Reuters) - Maryland, Delaware and ...",1
...,...,...
7632,"After they were crushed in the 2012 election, ...",0
6480,Just when we thought that Sarah Palin was out ...,0
2515,MOSCOW (Reuters) - Russia warned on Wednesday ...,1
5924,During a segment of HBO s Real Time with Bill ...,0


In [18]:
def wordopt(text):
    text = text. lower()
    text = re.sub('\[.*?]','',text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.S+','',text)
    text = re.sub('<.*?>+','',text)
    text = re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text = re.sub('\n','',text)
    text = re.sub('\w*\d\w*','',text)
    return text


In [19]:
data['text'] = data['text'].apply(wordopt)

In [20]:
X=data['text']
y=data['class']

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
vectorizer = TfidfVectorizer()

In [24]:
xv_train = vectorizer.fit_transform(X_train)
xv_test = vectorizer.transform(X_test)

In [25]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [26]:
pred_Ir = LR.predict(xv_test)

In [27]:
print(LR.score(xv_test, y_test))
print(classification_report(y_test, pred_Ir))

0.985204991087344
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5823
           1       0.98      0.99      0.98      5397

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [28]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [29]:
pred_dt = DT.predict(xv_test)

In [30]:
print(DT.score(xv_test, y_test))
print(classification_report(y_test, pred_dt))

0.996078431372549
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5823
           1       1.00      1.00      1.00      5397

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
RFC = RandomForestClassifier()

In [33]:
RFC.fit(xv_train,y_train)

RandomForestClassifier()

In [34]:
pred_rfc = RFC.predict(xv_test)

In [35]:
print(accuracy_score(pred_rfc,y_test))
print(classification_report(pred_rfc,y_test))

0.9887700534759358
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5827
           1       0.99      0.99      0.99      5393

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [36]:
train_pred = RFC.predict(xv_train)
print("Training Accuracy : " + str(accuracy_score(y_train,train_pred)))
print("Testing Accuracy : " + str(accuracy_score(y_test,pred_rfc)))

Training Accuracy : 0.999970289381425
Testing Accuracy : 0.9887700534759358


In [37]:
def output_label(n):
    if n==0:
        return "Fake News"
    elif n==1:
        return "Not A Fake News"

In [38]:
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorizer.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)
    return print("\nLR Prediction: {} \nDT Prediction: {} \nRFC Prediction: {}".format(output_label(pred_LR[0]),
                                                                                         output_label(pred_DT[0]),
                                                                                         output_label(pred_RFC[0])))

 

In [39]:
news = str(input())
manual_testing(news)


LR Prediction: Not A Fake News 
DT Prediction: Fake News 
RFC Prediction: Not A Fake News
